In [1]:
# Install necessary packages if not already installed:
# pip install ollama beautifulsoup4 requests

# Imports
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama  # Ollama's official Python client

# Function to fetch and parse website content 
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

class Website:
    def __init__(self, url):
        """
        Fetch and parse the webpage using BeautifulSoup.
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        # Remove unwanted elements
        for tag in soup(["script", "style", "img", "input"]):
            tag.decompose()

        self.text = soup.get_text(separator="\n", strip=True)

# Prompt Templates
system_prompt = "You are an assistant that summarizes website content. Ignore navigation, ads, or unrelated content. Respond concisely in Markdown."

def user_prompt_for(website):
    return f"""You are viewing a website titled: {website.title}.
The content is as follows:
{website.text}

Please summarize this website clearly in Markdown format.
"""

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# Summarize Function using Ollama
def summarize(url):
    website = Website(url)
    messages = messages_for(website)

    # Call Ollama local server
    response = ollama.chat(
        model='llama3',  # Change to your local model if needed
        messages=messages
    )
    return response['message']['content']

# Display summary in Markdown (for Jupyter Notebooks)
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

# Example Run
display_summary("https://edwarddonner.com")  # Change URL to test other sites


Here is a summary of the website:
* Edward Donner, co-founder and CTO of Nebula.io, introduces himself as a code writer, LLM experimenter, DJ, and amateur electronic music producer.
* He mentions his previous startup, untapt, which was acquired in 2021.
* His company, Nebula.io, uses AI to help people discover their potential and pursue their reason for being. They have patented their matching model and have happy customers with tons of press coverage.
* The website also lists recent posts, including courses on LLM expertise and AI engineering, as well as a workshop on agent-based AI.

Here is the summary in Markdown format:
```markdown
Introduction to Edward Donner
==========================

Edward Donner introduces himself as a code writer, LLM experimenter, DJ, and amateur electronic music producer. He mentions his previous startup, untapt, which was acquired in 2021.

About Nebula.io
----------------

Nebula.io uses AI to help people discover their potential and pursue their reason for being. They have patented their matching model and have happy customers with tons of press coverage.

Recent Posts
-------------

* Courses on LLM expertise and AI engineering
* LLM Workshop – Hands-on with Agents – resources
```